每个卷积块中的基本单元是一个卷积层、一个sigmoid激活函数和平均汇聚层。请注意，虽然ReLU和最大汇聚层更有效，但它们在20世纪90年代还没有出现。每个卷积层5*5使用卷积核和一个sigmoid激活函数。这些层将输入映射到多个二维特征输出，通常同时增加通道的数量。第一卷积层有6个输出通道，而第二个卷积层有16个输出通道。每个2*2池操作（步骤2）通过空间下采样将维数减少4倍。卷积的输出形状由批量大小、通道数、高度、宽度决定。

为了将卷积块的输出传递给稠密块，我们必须在小批量中展平每个样本。换言之，我们将这个四维输入转换成全连接层所期望的二维输入。这里的二维表示的第一个维度索引小批量中的样本，第二个维度给出每个样本的平面向量表示。LeNet的稠密块有三个全连接层，分别有120、84和10个输出。因为我们在执行分类任务，所以输出层的10维对应于最后输出结果的数量。

通过下面的LeNet代码，你会相信用深度学习框架实现此类模型非常简单。我们只需要实例化一个Sequential块并将需要的层连接在一起。

In [16]:
import torch
from torch import nn
from d2l import torch as d2l

net=nn.Sequential(
    nn.Conv2d(1,6,kernel_size=5,padding=2),nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2,stride=2),
    nn.Conv2d(6,16,kernel_size=5),nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2,stride=2),
    nn.Flatten(),
    nn.Linear(16*5*5,120),nn.Sigmoid(),
    nn.Linear(120,84),nn.Sigmoid(),
    nn.Linear(84,10)
)

http://zh.d2l.ai/chapter_convolutional-neural-networks/lenet.html#img-lenet-vert

In [17]:
#稍微测试一下
X=torch.rand(size=(1,1,28,28),dtype=torch.float32)
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

Conv2d output shape:	 torch.Size([1, 6, 28, 28])
Sigmoid output shape:	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape:	 torch.Size([1, 6, 14, 14])
Conv2d output shape:	 torch.Size([1, 16, 10, 10])
Sigmoid output shape:	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape:	 torch.Size([1, 16, 5, 5])
Flatten output shape:	 torch.Size([1, 400])
Linear output shape:	 torch.Size([1, 120])
Sigmoid output shape:	 torch.Size([1, 120])
Linear output shape:	 torch.Size([1, 84])
Sigmoid output shape:	 torch.Size([1, 84])
Linear output shape:	 torch.Size([1, 10])


In [18]:
# 加载数据集测试
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size=batch_size)

为了进行评估，我们需要对 3.6节中描述的evaluate_accuracy函数进行轻微的修改。 由于完整的数据集位于内存中，因此在模型使用GPU计算数据集之前，我们需要将其复制到显存中

In [19]:
def evaluate_accuracy_gpu(net,data_iter,device=None):
    if isinstance(net,nn.Module):
        net.eval()
        if not device:
            device=next(iter(net.parameters())).device
        metric=d2l.Accumulator(2)   #用于对多个变量进行累加。 在上面的evaluate_accuracy函数中， 我们在Accumulator实例中创建了2个变量， 分别用于存储正确预测的数量和预测的总数量。
        with torch.no_grad():
            for x,y in data_iter:
                if isinstance(x,list):
                    x=[x.to(device) for x in x]
                else:
                    x=x.to(device)     #.to(device)  转到GPU上运行  ==.cuda(x)
                y=y.to(device)
                metric.add(d2l.accuracy(net(x),y),y.numel())  #y.numel() 返回张量的总个数
        return metric[0]/metric[1]   #预测值/真实值

初始化模型参数  Xavier  http://zh.d2l.ai/chapter_multilayer-perceptrons/numerical-stability-and-init.html#subsec-xavier

们使用交叉熵损失函数和小批量随机梯度下降

In [23]:
def train_ch6(net,train_iter,test_iter,num_epochs,lr,device):
    '''
    训练模型
    :param net:  神经网络
    :param train_iter:训练数据集
    :param test_iter: 测试数据集
    :param num_epochs: 迭代
    :param lr:
    :param device: GPU
    :return:
    '''
    def init_weights(m):
        if type(m)==nn.Linear or type(m)==nn.Conv2d:  #采用的网络构建模型
            nn.init.xavier_uniform_(m.weight)#通常m这个类里包含weight bias  #均匀分布  uniform
    net.apply(init_weights) #对模型参数进行初始化
    print('training on:',device)
    net.to(device)
    optimizer=torch.optim.SGD(net.parameters(),lr=lr)  #模型每次反向传导都会给各个可学习参数p计算出一个偏导数g_{t}，用于更新对应的参数p。通常偏导数g_{t}不会直接作用到对应的可学习参数p上，而是通过优化器做一下处理，得到一个新的值\widehat{g}_t，处理过程用函数F表示（不同的优化器对应的F的内容不同），即\widehat{g}_t=F(g_{t})，然后和学习率lr一起用于更新可学习参数p，即p=p-\widehat{g}_t*lr。
    loss=nn.CrossEntropyLoss() #平方损失函数
    animator=d2l.Animator(xlabel='epoch',xlim=[1,num_epochs],legend=['train_loss','train acc','test_acc']) #绘图已封装
    timer,num_batches=d2l.Timer(),len(train_iter)
    for epoch in range(num_batches):
        metric=d2l.Accumulator(3)
        net.train()
        for i,(X,y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X,y=X.to(device),y.to(device)
            y_hat=net(X)
            l=loss(y_hat,y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l*X.shape[0],d2l.accuracy(y_hat,y),X.shape[0])
                #为了计算精度，我们执行以下操作。 首先，如果y_hat是矩阵，那么假定第二个维度存储每个类的预测分数。 我们使用argmax获得每行中最大元素的索引来获得预测类别。 然后我们将预测类别与真实y元素进行比较。 由于等式运算符“==”对数据类型很敏感， 因此我们将y_hat的数据类型转换为与y的数据类型一致。 结果是一个包含0（错）和1（对）的张量。 最后，我们求和会得到正确预测的数量。 accuracy()
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                            (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
      f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
      f'on {str(device)}')




In [25]:
lr,num_epochs=0.9,10
train_ch6(net,train_iter,test_iter,num_epochs,lr,d2l.try_gpu())

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.